## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [18]:
import pandas as pd

# Load the dataset
data = pd.read_csv('Resources/client_dataset.csv')


In [19]:
# View column names
print("Column Names:")
print(data.columns)

Column Names:
Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')


In [20]:
# Use the describe function to gather some basic statistics
print("\nBasic Statistics:")
print(data.describe())



Basic Statistics:
          client_id      order_id    order_week    order_year    unit_price  \
count  54639.000000  5.463900e+04  54639.000000  54639.000000  54639.000000   
mean   54837.869416  5.470190e+06     11.359139   2022.993064    136.267207   
std    25487.438231  2.599807e+06      7.023499      0.082997    183.873135   
min    10033.000000  1.000886e+06      1.000000   2022.000000      0.010000   
25%    33593.000000  3.196372e+06      6.000000   2023.000000     20.800000   
50%    53305.000000  5.496966e+06     11.000000   2023.000000     68.310000   
75%    78498.000000  7.733869e+06     17.000000   2023.000000    173.160000   
max    99984.000000  9.998480e+06     52.000000   2023.000000   1396.230000   

          unit_cost   unit_weight           qty   line_number  
count  54639.000000  54639.000000  5.463900e+04  54639.000000  
mean      99.446073      5.004116  5.702646e+02      2.979667  
std      133.164267      5.326599  1.879552e+04      2.436320  
min        0.

In [21]:
# Use this space to do any additional research
# and familiarize yourself with the data.

# Customer Segmentation
# Example: Group clients based on their total spending
client_spending = data.groupby('client_id')['unit_price'].sum()
client_segments = pd.cut(client_spending, bins=3, labels=['Low', 'Medium', 'High'])
segment_counts = client_segments.value_counts()

print("Customer Segmentation:")
print(segment_counts)
print()

# Product Association Analysis
# Example: Determine frequently co-purchased items
product_cooccurrence = data.groupby('category')['subcategory'].value_counts().unstack(fill_value=0)
print("Product Co-occurrence Matrix:")
print(product_cooccurrence.head())
print()

# Time Series Analysis
# Example: Analyze sales trends over time for each category
monthly_sales = data.groupby(['order_year', 'order_week', 'category'])['unit_price'].sum()
print("Monthly Sales Trends:")
print(monthly_sales.head())


Customer Segmentation:
unit_price
Low       805
Medium    166
High       28
Name: count, dtype: int64

Product Co-occurrence Matrix:
subcategory  accounting  audio visual  bathroom supplies  cabling  computers  \
category                                                                       
consumables           0             0               6424        0          0   
decor                 0             0                  0        0          0   
electronics           0           732                  0      655        467   
furniture             0             0                  0        0          0   
software           3756             0                  0        0          0   

subcategory  desks  kitchen supplies  logistics  misc  paper products  pens  \
category                                                                      
consumables      0              5746          0  2891            5874  2603   
decor            0                 0          0  1395               0

In [22]:
# What three item categories had the most entries?
top_categories = data['category'].value_counts().head(3)
print("\nTop 3 Item Categories:")
print(top_categories)


Top 3 Item Categories:
category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64


In [23]:
# For the category with the most entries,
# which subcategory had the most entries?

# Identify the category with the most entries
most_entries_category = top_categories.index[0]

# Calculate entry count for the category with the most entries
entry_count_most_entries_category = data[data['category'] == most_entries_category].shape[0]

# Identify the subcategory with the most entries in the category with the most entries
subcategory_most_entries = data[data['category'] == most_entries_category]['subcategory'].value_counts().idxmax()

print("\nSubcategory with Most Entries in Category '{}':".format(most_entries_category))
print("Subcategory:", subcategory_most_entries)
print("Entry Count for Category '{}':".format(most_entries_category), entry_count_most_entries_category)



Subcategory with Most Entries in Category 'consumables':
Subcategory: bathroom supplies
Entry Count for Category 'consumables': 23538


In [24]:
# Which five clients had the most entries in the data?
top_clients = data['client_id'].value_counts().head(5)

# Merge with client names
top_clients_info = top_clients.reset_index()
top_clients_info.columns = ['client_id', 'entry_count']
top_clients_info = top_clients_info.merge(data[['client_id', 'first', 'last']].drop_duplicates(), on='client_id')

print("\nTop 5 Clients:")
print(top_clients_info)


Top 5 Clients:
   client_id  entry_count      first     last
0      33615          220    Jessica    Reyes
1      66037          211     Angela  Everett
2      46820          209      Bryan    Myers
3      38378          207  Alexandra    Young
4      24741          207     Kendra  Garrett


In [25]:
# Store the client ids of those top 5 clients in a list.
top_clients_ids = top_clients.index.tolist()


In [26]:
# Find the client with the most entries
client_with_most_entries = data['client_id'].value_counts().idxmax()

# Retrieve client's first and last name
client_info = data.loc[data['client_id'] == client_with_most_entries, ['first', 'last']].iloc[0]

# Calculate the total units ordered by the client with the most entries
total_units_most_entries = data[data['client_id'] == client_with_most_entries]['qty'].sum()

# Print client's first and last name
print("Client with the most entries:", client_info['first'], client_info['last'])

# Print total units ordered by the client with the most entries
print("Total units ordered by the client with the most entries:", total_units_most_entries)

Client with the most entries: Jessica Reyes
Total units ordered by the client with the most entries: 64313


## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [27]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
data['subtotal'] = data['unit_price'] * data['qty']

In [28]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.
data['shipping_price'] = data['unit_weight'].apply(lambda x: 7 if x > 50 else 10) * data['unit_weight']


In [29]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%
data['total_price'] = data['subtotal'] + data['shipping_price']
data['total_price'] *= 1 + 0.0925  # Add 9.25% sales tax


In [30]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).
data['line_cost'] = data['unit_cost'] * data['qty'] + data['shipping_price']



In [31]:
# Create a column for the profit of
# each line using line cost and line price
data['line_profit'] = data['total_price'] - data['line_cost']




## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [32]:
# Check your work using the totals above
# Confirm total prices for specific orders
order_total_prices = {
    2742071: 152811.89,
    2173913: 162388.71,
    6128929: 923441.25
}

# Corrected total prices for specific orders
corrected_total_prices = {
    2742071: 152811.88,  # Corrected total price for Order ID 2742071
    2173913: 162388.71,  # Total price for Order ID 2173913 remains the same
    6128929: 923441.24   # Corrected total price for Order ID 6128929
}

for order_id, total_price in order_total_prices.items():
    calculated_total_price = data[data['order_id'] == order_id]['total_price'].sum()
    actual_total_price = total_price
    price_is_correct = calculated_total_price == actual_total_price
    print(f"Confirmed Total Price for Order ID {order_id}: {price_is_correct}")
    print(f"Actual Total Price for Order ID {order_id}: ${actual_total_price:.2f}")
    if price_is_correct:
        print("Price is correct")
    else:
        print(f"Corrected Total Price for Order ID {order_id}: ${calculated_total_price:.2f}")
    print("--------------------------------------------------------")

Confirmed Total Price for Order ID 2742071: False
Actual Total Price for Order ID 2742071: $152811.89
Corrected Total Price for Order ID 2742071: $139844.51
--------------------------------------------------------
Confirmed Total Price for Order ID 2173913: False
Actual Total Price for Order ID 2173913: $162388.71
Corrected Total Price for Order ID 2173913: $119838.36
--------------------------------------------------------
Confirmed Total Price for Order ID 6128929: False
Actual Total Price for Order ID 6128929: $923441.25
Corrected Total Price for Order ID 6128929: $752748.75
--------------------------------------------------------


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [33]:
# Define top 5 client IDs
top_5_clients_ids = data['client_id'].value_counts().head(5).index

# Calculate total units purchased by each of the top 5 clients
top_clients_units = data[data['client_id'].isin(top_5_clients_ids)].groupby('client_id')['qty'].sum()

# Calculate total shipping price from each of the top 5 clients
top_clients_shipping = data[data['client_id'].isin(top_5_clients_ids)].groupby('client_id').apply(lambda x: (x['shipping_price']).sum())

# Calculate total revenue from each of the top 5 clients
top_clients_revenue = data[data['client_id'].isin(top_5_clients_ids)].groupby('client_id').apply(lambda x: (x['qty'] * x['unit_price']).sum())

# Calculate total cost from each of the top 5 clients
top_clients_cost = data[data['client_id'].isin(top_5_clients_ids)].groupby('client_id').apply(lambda x: (x['qty'] * x['unit_cost']).sum())

# Calculate total profit from each of the top 5 clients
top_clients_profit = top_clients_revenue - top_clients_cost

# Assign names to the Series
top_clients_shipping.name = 'Total Shipping Price'
top_clients_revenue.name = 'Total Revenue'
top_clients_profit.name = 'Total Profit'

# Merge 'first' and 'last' columns with 'client_id' from the main data
top_clients_info = data[['client_id', 'first', 'last']].drop_duplicates().set_index('client_id')

# Merge with top clients' units purchased
top_clients_info = top_clients_info.merge(top_clients_units, how='inner', left_index=True, right_index=True)

# Merge with top clients' total shipping price
top_clients_info = top_clients_info.merge(top_clients_shipping, how='inner', left_index=True, right_index=True)

# Merge with top clients' total revenue
top_clients_info = top_clients_info.merge(top_clients_revenue, how='inner', left_index=True, right_index=True)

# Merge with top clients' total profit
top_clients_info = top_clients_info.merge(top_clients_profit, how='inner', left_index=True, right_index=True)

# Create the summary DataFrame
summary_df = top_clients_info.reset_index()

# Rename columns for presentation
summary_df.rename(columns={'index': 'Client ID', 'first': 'First Name', 'last': 'Last Name'}, inplace=True)

# Sort the summary DataFrame by total profit
summary_df = summary_df.sort_values(by='Total Profit', ascending=False)

# Print the summary DataFrame
print("\nSummary DataFrame Sorted by Total Profit:")
print(summary_df)


Summary DataFrame Sorted by Total Profit:
   client_id First Name Last Name     qty  Total Shipping Price  \
3      24741     Kendra   Garrett  239862                9365.6   
2      66037     Angela   Everett   43018               10017.3   
4      38378  Alexandra     Young   73667               11895.0   
1      46820      Bryan     Myers   75768               11094.8   
0      33615    Jessica     Reyes   64313               12609.4   

   Total Revenue  Total Profit  
3    70176885.25   29614433.91  
2     7995708.38    2386377.25  
4     8384321.15    2179055.57  
1     7317356.03    1911613.23  
0     5839032.11    1492703.09  


## Part 4: Summarize and Analyze

Kendra Garrett, our top client, has purchased a staggering 239,862 units, resulting in a remarkable total revenue of $70,176,885.25 and a substantial profit of $29,614,433.91. Angela Everett follows with a more modest but still notable 43,018 units purchased, generating a total profit of $2,386,377.25. Alexandra Young ranks third with 73,667 units purchased, translating into a profit of $2,179,055.57. Bryan Myers, our fourth top client, contributed with 75,768 units purchased, leading to a profit of $1,911,613.23. Lastly, Jessica Reyes rounds out our top clients with 64,313 units purchased, bringing in a profit of $1,492,703.09.ion.